## Face Detection AI Project

In [2]:
! omz_downloader --name face-detection-adas-0001 --precision FP32

################|| Downloading face-detection-adas-0001 ||################

========== Downloading C:\BrainAI\face-detection\intel\face-detection-adas-0001\FP32\face-detection-adas-0001.xml
... 100%, 228 KB, 69 KB/s, 3 seconds passed

========== Downloading C:\BrainAI\face-detection\intel\face-detection-adas-0001\FP32\face-detection-adas-0001.bin
... 24%, 1024 KB, 744 KB/s, 1 seconds passed
... 49%, 2048 KB, 1272 KB/s, 1 seconds passed
... 74%, 3072 KB, 1872 KB/s, 1 seconds passed
... 99%, 4096 KB, 2221 KB/s, 1 seconds passed
... 100%, 4113 KB, 2230 KB/s, 1 seconds passed



In [3]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path

In [4]:
core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU']

In [5]:
core = ov.Core()

model = core.read_model(model='./models/face-detection-adas-0001.xml')
face_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)

print("Input layer shape: ", face_input_layer.shape)
print("Output layer shape:", face_output_layer.shape)

Input layer shape:  [1,3,384,672]
Output layer shape: [1,1,200,7]


In [9]:
frame = cv2.imread("images/test.jpeg")
resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

In [10]:
face_output = face_model([input_frame])[face_output_layer]

In [11]:
def DrawBoundingBoxes(output, image, conf):

    canvas = image.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h]) # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box.astype("int")  # xmin, ymin, xmax, ymax에 상자 위치 값 지정

        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
    
   
    return canvas

In [29]:
canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)
cv2.imshow("Face", canvas)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 350
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

In [28]:
background = "./images/Bacground.jpg"  #사용할 배경화면 경로
canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)  
bg = cv2.imread(background)

deployment = AddBackground(canvas, bg)
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
background = "./images/Background.jpg"
bg = cv2.imread(background)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)    
    
    face_output = face_model([input_frame])[face_output_layer]

    canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)
    deployment = AddBackground(canvas, bg)
    
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()